# Pair Programming Intro Regresión Lineal

En el pair programming de hoy debéis usar el csv que guardastéis cuando hicistéis el 

pairprgramming de codificicación (este csv debería tener las variables **estadandarizas**).

El **objetivo** de este pairprogramming es que hagáis vuestro primer modelo de machine learning. Para eso recordad que:

- Deberéis separar la X y la y.
- Deberéis crear el train y el test.
- Ajustar vuestro modelo.

In [1]:
# Tratamiento de datos
# -----------------------------------------------------------------------
import numpy as np
import pandas as pd

# Gráficos
# ------------------------------------------------------------------------------
import matplotlib.pyplot as plt
import seaborn as sns


#  Modelado y evaluación
# ------------------------------------------------------------------------------
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression


#  Crossvalidation
# ------------------------------------------------------------------------------
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn import metrics

In [2]:
df=pd.read_csv('data/08-primas_categorizadas_normalizadas.csv',index_col=0)
df.head()

,index,age,sex,bmi,children,smoker,region,charges_Sklearn,fumadores,x0_northeast,x0_northwest,x0_southeast,x0_southwest
0,-1.654590,-0.683871,male,-1.697491,-1.011604,no,northwest,0.045128,0,0.0,1.0,0.0,0.0
1,-1.639514,1.268419,female,-1.120191,-1.011604,no,northwest,0.203687,0,0.0,1.0,0.0,0.0
2,-1.634489,1.413033,female,-1.037325,-1.011604,yes,southeast,0.178221,1,0.0,0.0,1.0,0.0
3,-1.626951,-1.117713,male,1.879556,-1.011604,yes,southeast,0.447938,1,0.0,0.0,1.0,0.0
4,-1.614388,-0.900792,male,0.621835,-1.011604,yes,southwest,0.384541,1,0.0,0.0,0.0,1.0
